In [2]:
# training_set = dataset_train.iloc[:, 1:3].values

# X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 2)) 
# Use this to add another column to the initial matrix. When i get to the np.reshape function i used: 

# for i in range(60, 1258):
#     X_train.append(training_set_scaled[i-60:i, 0:2])

In [3]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Input, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [4]:
dataset_train = pd.read_csv('Google_Stock_Price_Train.csv')
training_set = dataset_train.iloc[:, 1:2].values

In [5]:
training_set

array([[325.25],
       [331.27],
       [329.83],
       ...,
       [793.7 ],
       [783.33],
       [782.75]])

In [6]:
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

In [7]:
training_set_scaled

array([[0.08581368],
       [0.09701243],
       [0.09433366],
       ...,
       [0.95725128],
       [0.93796041],
       [0.93688146]])

In [8]:
X_train = []
y_train = []

for i in range(60, len(training_set)):
    X_train.append(training_set_scaled[i-60:i, 0])   # i = 60 - 60 = 0 : 60 [0:59]
    y_train.append(training_set_scaled[i, 0]) # treinar [60] para prevê

X_train, y_train = np.array(X_train), np.array(y_train)

In [9]:
pd.DataFrame(X_train)

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,0.085814,0.097012,0.094334,0.091562,0.079842,0.064328,0.058542,0.065686,0.061091,0.066393,...,0.052143,0.056124,0.058189,0.065407,0.068830,0.072438,0.079935,0.078466,0.080345,0.084977
1,0.097012,0.094334,0.091562,0.079842,0.064328,0.058542,0.065686,0.061091,0.066393,0.061426,...,0.056124,0.058189,0.065407,0.068830,0.072438,0.079935,0.078466,0.080345,0.084977,0.086279
2,0.094334,0.091562,0.079842,0.064328,0.058542,0.065686,0.061091,0.066393,0.061426,0.074745,...,0.058189,0.065407,0.068830,0.072438,0.079935,0.078466,0.080345,0.084977,0.086279,0.084716
3,0.091562,0.079842,0.064328,0.058542,0.065686,0.061091,0.066393,0.061426,0.074745,0.027978,...,0.065407,0.068830,0.072438,0.079935,0.078466,0.080345,0.084977,0.086279,0.084716,0.074541
4,0.079842,0.064328,0.058542,0.065686,0.061091,0.066393,0.061426,0.074745,0.027978,0.023793,...,0.068830,0.072438,0.079935,0.078466,0.080345,0.084977,0.086279,0.084716,0.074541,0.078838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1193,0.934445,0.924939,0.921069,0.924381,0.930482,0.929905,0.931133,0.927506,0.944155,0.938760,...,0.931766,0.941141,0.957623,0.964134,0.964023,0.969715,0.950778,0.962944,0.961232,0.954759
1194,0.924939,0.921069,0.924381,0.930482,0.929905,0.931133,0.927506,0.944155,0.938760,0.934035,...,0.941141,0.957623,0.964134,0.964023,0.969715,0.950778,0.962944,0.961232,0.954759,0.952043
1195,0.921069,0.924381,0.930482,0.929905,0.931133,0.927506,0.944155,0.938760,0.934035,0.934835,...,0.957623,0.964134,0.964023,0.969715,0.950778,0.962944,0.961232,0.954759,0.952043,0.951633
1196,0.924381,0.930482,0.929905,0.931133,0.927506,0.944155,0.938760,0.934035,0.934835,0.931394,...,0.964134,0.964023,0.969715,0.950778,0.962944,0.961232,0.954759,0.952043,0.951633,0.957251


In [10]:
# (X_train_shape[0] (número de linhas)  X_train_shape[1] (número de colunas))
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [11]:
# Eixo 0 (1198): Representa o número de janelas criadas (uma para cada dia a partir do 60º). relacionado ao y_train
# Eixo 1 (60): Representa os 60 dias consecutivos dentro de cada janela.
# Eixo 2 (1): Representa o canal (único neste caso, com o valor normalizado).
for i, size in enumerate(X_train.shape):
    print(f"Eixo {i}: Tamanho = {size}")

Eixo 0: Tamanho = 1198
Eixo 1: Tamanho = 60
Eixo 2: Tamanho = 1


In [12]:
regressor = Sequential([
    
    # Camada de entrada: define a forma da entrada do modelo (número de timesteps e features).
    # O formato de entrada é baseado em X_train, onde X_train.shape[1] é o número de timesteps e 1 é o número de features.
    Input(shape=(X_train.shape[1], 1)),
    
    # Camada LSTM (Long Short-Term Memory):
    # 'units=50' define o número de unidades (neurônios) na camada LSTM.
    # 'return_sequences=True' significa que esta camada retornará a sequência de saídas, não apenas a saída final,
    # permitindo que a próxima camada LSTM ou RNN possa continuar processando a sequência.
    LSTM(units=50, return_sequences=True),
    Dropout(0.2),
    LSTM(units=50, return_sequences=True),
    Dropout(0.2),
    LSTM(units=50, return_sequences=False),
    Dropout(0.2),
    Dense(units = 1)
])

In [13]:
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [14]:
early_stopping = EarlyStopping( monitor='loss', patience=10, restore_best_weights=True)

In [15]:
regressor.fit(X_train, y_train, epochs = 100, batch_size = 32, callbacks=[early_stopping])

Epoch 1/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - loss: 0.1114
Epoch 2/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.0067
Epoch 3/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.0056
Epoch 4/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0050
Epoch 5/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.0043
Epoch 6/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.0042
Epoch 7/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0041
Epoch 8/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0043
Epoch 9/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0040
Epoch 10/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0040
Epoch 11/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0044
Epoch 12/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.0037
Epoch 13/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0040
Epoch 14/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0038
Epoch 15/100
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - lo